# Gradient Boosting

### Basic single model:
Adapted from [here](http://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html).

In [ ]:
from sklearn.datasets import load_boston
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
boston = load_boston()

X, y = shuffle(boston.data, boston.target, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [ ]:
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

gb_r = ensemble.GradientBoostingRegressor(**params)

gb_r.fit(X_train, y_train)
mse = mean_squared_error(y_test, gb_r.predict(X_test))
print("MSE: %.4f" % mse)

In [ ]:
% matplotlib inline

# compute test set deviance
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, y_pred in enumerate(gb_r.staged_predict(X_test)):
    test_score[i] = gb_r.loss_(y_test, y_pred)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, gb_r.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')

In [ ]:
feature_importance = gb_r.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, boston.feature_names[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

## Grid search:

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': range(450, 551, 50),
              'max_depth': range(1, 12, 5),
              'min_samples_split': [2],
              'learning_rate': np.arange(0.01, .22, .1),
              'loss': ['ls']}

gb_r = GridSearchCV(GradientBoostingRegressor(), param_grid)
gb_r.fit(X_train, y_train)

In [ ]:
sorted(gb_r.cv_results_.keys())

In [ ]:
gb_r.cv_results_["mean_test_score"]

In [ ]:
best_index = np.argmax(gb_r.cv_results_["mean_test_score"])

In [ ]:
gb_r.cv_results_["params"]

In [ ]:
print(gb_r.cv_results_["params"][best_index])
print()
print(max(gb_r.cv_results_["mean_test_score"]))